In [2]:
# | default_exp client

In [ ]:
# | export 


import json
from typing import Dict, Any, List
from enum import Enum
from dataclasses import dataclass
from rich.console import Console
from rich.panel import Panel
from rich.align import Align
from rich.panel import Panel
from agentic.tools.manager import ToolManager
from agentic.core.agent import Agent, AgentConfig
from agentic.configs.loader import get_model_config, get_settings_config, get_tools_config, get_reasoning_config


class RequestComplexity(Enum):
    SIMPLE = "simple"
    MODERATE = "moderate"
    COMPLEX = "complex"


@dataclass
class AnalysisResult:
    complexity: RequestComplexity
    confidence: float
    reasoning: str
    requires_planning: bool
    requires_debate: bool
    suggested_tools: List[str]


class BuddyClient:
    """Production-grade AI assistant with intelligent request routing"""
    
    def __init__(self, model: str = None, base_url: str = None, api_key: str = None):
        # Load configurations
        self.model_config = get_model_config()
        self.settings_config = get_settings_config()
        
        # Initialize tool manager
        self.tool_manager = ToolManager()
        
        # Create agent config with routing instructions
        agent_config = AgentConfig(
            name="BuddyAgent",
            instructions = """
You are Buddy, a helpful and reliable AI assistant designed to support users by performing tasks and answering questions using a set of integrated tools, including custom utilities and MCP-based servers for filesystem operations, code execution, and more.

🔧 TOOL USAGE PRINCIPLES:
- Leverage tools proactively when they provide a more efficient, accurate, or comprehensive solution than manual reasoning alone.
- Before invoking any tool that executes code or modifies the environment, verify dependencies (e.g., required packages or libraries) and confirm they are available.
- If dependencies are missing, clearly list them, explain their necessity, and request explicit user permission before proceeding with installation or setup.
- Respect user consent: Only act if approved; otherwise, suggest alternatives or gracefully decline.
- For tools involving external resources (e.g., filesystem or shell access), ensure operations are scoped to safe, user-authorized directories or environments to prevent unintended changes.

⚙️ OPTIMIZATION GUIDELINES:
- Analyze each user request to identify the optimal path: Prioritize tools that minimize steps, reduce errors, and deliver high-quality results.
- Use tools for complex computations, data processing, file handling, or system interactions where direct execution outperforms textual simulation.
- Focus on performance, security, and relevance: Select the most suitable tool based on the task's nature, avoiding overkill for simple queries.

📌 TASK ROUTING:
- For requests involving comparisons, evaluations, trade-offs, or debates, route to specialized analysis tools.
- For multi-step projects, planning, or structured implementations (e.g., building, designing, or developing), employ planning or orchestration tools.
- For all other tasks—such as reading/writing data, executing commands, interpreting code, or querying resources—select the appropriate tool dynamically based on context.

🎯 RESPONSE BEHAVIOR:
- Always frame responses around the user's original intent, integrating tool outputs seamlessly into clear, actionable explanations.
- Summarize and interpret results meaningfully, avoiding raw dumps unless requested.
- Maintain transparency about tool usage without revealing internal mechanics, unless the user inquires.
- Adopt a conversational, empathetic tone: Be concise yet thorough, proactive in clarifications, and focused on user satisfaction.

💡 OVERALL GOAL:
Provide an intuitive, secure, and efficient experience—empowering users with tool-assisted insights while ensuring safety, consent, and optimal outcomes in every interaction.
            """
            ,

            model=model or self.model_config.get('name'),
            temperature=self.model_config.get('temperature', 0.7),
            max_tokens=self.model_config.get('max_tokens')
        )
        
        # Initialize single Agent for all operations
        self.agent = Agent(config=agent_config)
        
        # Initialize console
        self.console = Console()
        
        # Settings - force streaming to true
        self.auto_approve = self.settings_config.get('auto_approve', False)
        self.stream = True  # Always enable streaming
        self.debug = self.settings_config.get('debug', False)

        
    def execute_simple_request(self, request: str, suggested_tools: List[str]) -> Dict[str, Any]:
        """Execute simple requests directly with Agent and tools"""
        
        self.console.print(Panel(
            f"[bold green]Executing simple request with direct tool usage[/bold green]",
            title="[bold yellow]🚀 Simple Execution[/bold yellow]",
            border_style="yellow"
        ))
        
        try:
            # Use Agent directly - it handles tool selection and execution
            result = self.agent.run(
                request, 
                stream=self.stream,
                max_iterations=5
            )
            
            return {
                "success": True,
                "result": result.get("content", ""),
                "tool_calls": result.get("tool_calls", []),
                "execution_type": "simple"
            }
            
        except Exception as e:
            self.console.print(f"[bold red]❌ Simple execution failed:[/bold red] {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "execution_type": "simple"
            }
    
    def execute_complex_request(self, request: str, analysis: AnalysisResult) -> Dict[str, Any]:
        """Execute complex requests by routing to appropriate agents"""
        
        self.console.print(Panel(
            f"[warning]Complex request detected - routing to appropriate agent[/warning]\n"
            f"Complexity: {analysis.complexity.value}\n"
            f"Confidence: {analysis.confidence:.2f}\n"
            f"Reasoning: {analysis.reasoning}",
            title="🧠 Complex Execution"
        ))
        
        try:
            # Route to debate agent if debate is required
            if analysis.requires_debate:
                return self._execute_debate_phase(request)
            
            # Route to planner for moderate/complex planning tasks
            elif analysis.requires_planning:
                return self._execute_planning_phase(request)
            
            # Fallback to simple execution
            else:
                return self.execute_simple_request(request, analysis.suggested_tools)
                
        except Exception as e:
            self.console.print(f"[bold red]❌ Complex execution failed:[/bold red] {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "execution_type": "complex"
            }
    
    def _execute_planning_phase(self, request: str) -> Dict[str, Any]:
        """Execute planning phase using planner agent"""
        
        try:
            from agentic.agent.planner.executor import DynamicTaskExecutor
            
            executor = DynamicTaskExecutor(self.agent, self.console)
            result = executor.execute(request)
            
            # Display completion message
            self.console.print(Panel(
                "[bold green]✅ Planning and execution completed![/bold green]",
                title="Planning Results",
                border_style="green"
            ))
            
            return {
                "success": True,
                "result": result,
                "execution_type": "planning"
            }
            
        except Exception as e:
            self.console.print(f"[bold red]❌ Planning failed:[/bold red] {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "execution_type": "planning"
            }
    
    def _execute_debate_phase(self, request: str) -> Dict[str, Any]:
        """Execute debate phase using debate agent"""
        
        try:
            from agentic.agent.debater import create_debate
            import asyncio
            
            # Run debate asynchronously
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            
            try:
                result = loop.run_until_complete(
                    create_debate(
                        topic=f"Analysis: {request}",
                        context=f"User request: {request}",
                        max_rounds=2
                    )
                )
            finally:
                loop.close()
            
            # Display completion message
            self.console.print(Panel(
                "[bold green]✅ Debate analysis completed![/bold green]",
                title="Debate Results",
                border_style="green"
            ))
            
            return {
                "success": True,
                "result": result,
                "execution_type": "debate"
            }
            
        except Exception as e:
            self.console.print(f"[bold red]❌ Debate failed:[/bold red] {str(e)}")
            return {
                "success": False,
                "error": str(e),
                "execution_type": "debate"
            }
    
    def process_request(self, request: str) -> Dict[str, Any]:
        """Main request processing pipeline - single step execution"""
        
        self.console.print(Panel(
            f"[bold cyan]Processing:[/bold cyan] {request}",
            title="[bold blue]🤖 Buddy AI[/bold blue]",
            border_style="blue"
        ))
        
        try:
            # Single step execution - Agent decides routing internally
            result = self.agent.run(
                request, 
                stream=self.stream,
                max_iterations=5
            )
            
            return {
                "success": True,
                "result": result.get("content", ""),
                "tool_calls": result.get("tool_calls", []),
                "execution_type": "direct"
            }
            
        except Exception as e:
            error_msg = f"Request processing failed: {str(e)}"
            self.console.print(f"[red]❌ {error_msg}[/red]")
            return {"success": False, "error": error_msg}
    

    def interactive_session(self):
        """
        Start an interactive chat session with Buddy AI.
        """
        welcome_message = (
            "[bold green]👋 Welcome to [underline]Buddy AI[/underline]! 🤖[/bold green]\n\n"
            "[cyan]I’m your assistant for coding, writing, and everyday tasks.[/cyan]\n"
            "Ask me anything — from building projects to answering complex questions.\n\n"
            "[bold yellow]✨ Available Commands:[/bold yellow]\n\n"
            "  [bold red]➤ /quit[/bold red]     [dim]- Exit the session[/dim]\n"
            "  [bold magenta]➤ /clear[/bold magenta]    [dim]- Clear the chat history[/dim]\n"
        )

        # Center-align the content inside the panel
        # centered_content = Align.center(welcome_message, vertical="top")

        self.console.print(Panel(
            welcome_message,
            title="[bold blue]💬 Buddy AI Assistant[/bold blue]",
            border_style="bright_green",
            padding=(1, 4)  # (top/bottom, left/right) padding
        ))


        while True:
            try:            
                user_input = input("\n💬 You: ").strip()
                
                if not user_input:
                    continue
                
                # Handle commands
                if user_input.lower() in ['/quit', '/exit', 'quit', 'exit']:
                    self.console.print("[yellow]👋 Goodbye![/yellow]")
                    break
                elif user_input.lower() in ['/clear', 'clear']:
                    self.console.clear()
                    continue
                
                # Process the request
                result = self.process_request(user_input)
                
                if not result.get("success", False):
                    self.console.print(f"[red]❌ Error: {result.get('error', 'Unknown error')}[/red]")
                
            except KeyboardInterrupt:
                self.console.print("\n[yellow]👋 Goodbye![/yellow]")
                break
            except EOFError:
                self.console.print("\n[yellow]👋 Goodbye![/yellow]")
                break
            except Exception as e:
                self.console.print(f"[red]❌ Session error: {e}[/red]")


def main():
    """Main entry point"""
    try:
        client = BuddyClient()
        client.interactive_session()
    except Exception as e:
        print(f"❌ Failed to start Buddy AI: {e}")


